In [12]:
import sys
sys.path.append("../..")
import numpy as np
import joblib
data = joblib.load("../../../persist_data/atari_vf_fit.pkl")

In [15]:
policy = data["policy"]
mdp = data["mdp"]
paths = data["paths"]

In [20]:
n_train_paths = int(len(paths) * 0.7)
n_test_paths = len(paths) - n_train_paths

train_paths = paths[:n_train_paths]
test_paths = paths[n_train_paths:]

train_xs = np.concatenate([p["observations"] for p in train_paths])
train_ys = np.concatenate([p["returns"] for p in train_paths])
test_xs = np.concatenate([p["observations"] for p in test_paths])
test_ys = np.concatenate([p["returns"] for p in test_paths])

In [19]:
from rllab.baseline.gaussian_mlp_baseline import GaussianMLPBaseline
from rllab.misc.ext import AttrDict
from rllab.optimizer.penalty_lbfgs_optimizer import PenaltyLbfgsOptimizer

# Fix the initial baseline parameters
baseline = GaussianMLPBaseline(
    mdp,
)
init_params = baseline.get_param_values()
print np.linalg.norm(init_params)

9.32287


In [ ]:
# Let's inspect training loss first
baseline = GaussianMLPBaseline(
    mdp,
    use_trust_region=False
)
baseline.set_param_values(init_params)
baseline.fit(train_paths)
baseline._regressor.predict_log_likelihood(train_xs, train_ys).shape

2016-03-10 22:25:40.000414 PST | trying penalty=1.000...
2016-03-10 22:25:42.201615 PST | penalty 1.000000 => loss 1.615370, constraint 0.379543
2016-03-10 22:25:42.202127 PST | trying penalty=2.000...
2016-03-10 22:25:44.422307 PST | penalty 2.000000 => loss 1.861606, constraint 0.203668
2016-03-10 22:25:44.422833 PST | trying penalty=4.000...
2016-03-10 22:25:46.593398 PST | penalty 4.000000 => loss 2.155850, constraint 0.099573
2016-03-10 22:25:46.593926 PST | trying penalty=8.000...
2016-03-10 22:25:48.701308 PST | penalty 8.000000 => loss 2.480824, constraint 0.041323
2016-03-10 22:25:48.701820 PST | trying penalty=16.000...
2016-03-10 22:25:51.017007 PST | penalty 16.000000 => loss 2.774517, constraint 0.014566
2016-03-10 22:25:51.017529 PST | trying penalty=32.000...
2016-03-10 22:25:53.150825 PST | penalty 32.000000 => loss 2.994522, constraint 0.004405
